<a href="https://colab.research.google.com/github/brendanpshea/intro_to_networks/blob/main/Networks_08_DocumentingNetworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Rack Diagram

In [6]:
# @title
%%html
<!DOCTYPE html>
<html>
<head>
    <title>Rack Equipment Builder</title>
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: #f5f6fa;
            margin: 0;
            padding: 20px;
        }
        .main-container {
            display: flex;
            gap: 20px;
            max-width: 1200px;
            margin: 0 auto;
        }
        .left-panel {
            width: 320px;
            flex-shrink: 0;
        }
        .right-panel {
            flex-grow: 1;
        }
        .control-section {
            background: white;
            padding: 15px;
            margin-bottom: 15px;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
        }
        .control-section h3 {
            margin: 0 0 10px 0;
            color: #2c3e50;
            font-size: 16px;
        }
        .rack {
            width: 300px;
            border: 2px solid #2c3e50;
            background: linear-gradient(to right, #ecf0f1, #bdc3c7);
            padding: 10px;
            border-radius: 4px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            margin-bottom: 20px;
        }
        .rack-unit {
            height: 20px;
            border-bottom: 1px dashed #7f8c8d;
            margin: 1px 0;
            display: flex;
            align-items: center;
            justify-content: center;
            position: relative;
            transition: background-color 0.2s;
        }
        .rack-unit:hover {
            background-color: rgba(255, 255, 255, 0.5);
            cursor: pointer;
        }
        .rack-unit::before {
            content: attr(data-u);
            position: absolute;
            left: -30px;
            font-size: 12px;
            font-weight: bold;
            color: #34495e;
        }
        button {
            margin: 5px;
            padding: 8px 15px;
            cursor: pointer;
            border: none;
            border-radius: 4px;
            font-weight: 500;
            transition: all 0.2s;
            width: calc(100% - 10px);
            text-align: left;
        }
        button:hover {
            transform: translateY(-2px);
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
        }
        .selected {
            border: 2px solid #f1c40f;
            background: #2c3e50;
            color: white;
        }
        .equipment {
            width: 100%;
            color: white;
            text-align: center;
            border-radius: 3px;
            padding: 2px;
            font-size: 11px;
            box-shadow: 0 1px 3px rgba(0, 0, 0, 0.2);
            display: flex;
            flex-direction: column;
            justify-content: center;
        }
        .equipment-name {
            font-weight: bold;
            font-size: 10px;
            margin-bottom: 1px;
        }
        .equipment-type {
            font-size: 8px;
            opacity: 0.8;
        }
        /* Equipment type styles */
        .type-server { background: linear-gradient(to right, #3498db, #2980b9); }
        .type-storage { background: linear-gradient(to right, #2ecc71, #27ae60); }
        .type-network { background: linear-gradient(to right, #e74c3c, #c0392b); }
        .type-ups { background: linear-gradient(to right, #9b59b6, #8e44ad); }
        .type-custom { background: linear-gradient(to right, #f1c40f, #f39c12); }

        .config-description {
            background: white;
            padding: 15px;
            border-radius: 8px;
            margin-top: 20px;
        }
        .config-text {
            padding: 10px;
            background: #f8f9fa;
            border-radius: 4px;
            margin-top: 10px;
            white-space: pre-wrap;
            font-size: 14px;
            line-height: 1.5;
        }
    </style>
</head>
<body>
    <div class="main-container">
        <div class="left-panel">
            <div class="control-section">
                <h3>Sample Configurations</h3>
                <button onclick="loadSampleConfig('enterprise')" style="background: #27ae60; color: white;">Enterprise Setup</button>
                <button onclick="loadSampleConfig('smb')" style="background: #2980b9; color: white;">Small Business</button>
                <button onclick="loadSampleConfig('compute')" style="background: #8e44ad; color: white;">Compute Cluster</button>
                <button onclick="loadSampleConfig('storage')" style="background: #d35400; color: white;">Storage Cluster</button>
                <button onclick="clearRack()" style="background: #e74c3c; color: white; margin-top: 10px;">Clear Rack</button>
            </div>

            <div class="control-section">
                <h3>Servers</h3>
                <button onclick="selectEquipment('server', 1, 'Server')">1U Server</button>
                <button onclick="selectEquipment('server', 2, 'Server')">2U Server</button>
                <button onclick="selectEquipment('server', 4, 'Server')">4U Server</button>
            </div>

            <div class="control-section">
                <h3>Storage</h3>
                <button onclick="selectEquipment('storage', 2, 'Storage Array')">2U Storage</button>
                <button onclick="selectEquipment('storage', 4, 'Storage Array')">4U Storage</button>
            </div>

            <div class="control-section">
                <h3>Network</h3>
                <button onclick="selectEquipment('network', 1, 'Switch')">1U Switch</button>
                <button onclick="selectEquipment('network', 2, 'Router')">2U Router</button>
            </div>

            <div class="control-section">
                <h3>Power</h3>
                <button onclick="selectEquipment('ups', 2, 'UPS')">2U UPS</button>
                <button onclick="selectEquipment('ups', 4, 'UPS')">4U UPS</button>
            </div>
        </div>

        <div class="center-panel">
            <div id="rack" class="rack"></div>
        </div>

        <div class="right-panel">
            <div id="config-description" class="config-description" style="display: none;">
                <h3>Configuration Details</h3>
                <div id="config-text" class="config-text"></div>
            </div>
        </div>
    </div>

    <script>
        // Keep all the JavaScript the same as before
        const RACK_UNITS = 42;
        let selectedConfig = null;
        let rackContents = new Array(RACK_UNITS).fill(null);

        const configurations = {
            enterprise: {
                name: "Enterprise Production Setup",
                description: `This enterprise configuration emphasizes redundancy and high availability:
                • Redundant UPS units for power backup
                • Primary and backup storage arrays
                • Multiple application servers for load balancing
                • Dedicated database server
                • Redundant network switches
                • Management network separate from production

                Key design principles:
                • Critical infrastructure redundancy
                • Separated management network
                • Scalable application tier
                • Power equipment at bottom for stability
                • Network equipment at top for cable management`,
                config: [
                    { position: 2, type: 'ups', size: 2, name: 'Primary UPS' },
                    { position: 6, type: 'ups', size: 2, name: 'Backup UPS' },
                    { position: 10, type: 'storage', size: 4, name: 'Primary Storage Array' },
                    { position: 14, type: 'storage', size: 2, name: 'Backup Storage Array' },
                    { position: 18, type: 'server', size: 2, name: 'Database Server' },
                    { position: 20, type: 'server', size: 2, name: 'Application Server 1' },
                    { position: 22, type: 'server', size: 2, name: 'Application Server 2' },
                    { position: 24, type: 'server', size: 1, name: 'Web Server 1' },
                    { position: 25, type: 'server', size: 1, name: 'Web Server 2' },
                    { position: 26, type: 'server', size: 1, name: 'Management Server' },
                    { position: 30, type: 'network', size: 2, name: 'Core Router' },
                    { position: 40, type: 'network', size: 1, name: 'Management Switch' },
                    { position: 41, type: 'network', size: 1, name: 'Primary Switch' },
                    { position: 42, type: 'network', size: 1, name: 'Backup Switch' }
                ]
            },
            smb: {
                name: "Small Business Setup",
                description: `Optimized for small business needs with essential services:
                • Single UPS system
                • Consolidated storage
                • Virtualization host for multiple services
                • Basic networking

                Key design principles:
                • Cost-effective configuration
                • Simplified management
                • Essential redundancy only
                • Room for future growth`,
                config: [
                    { position: 2, type: 'ups', size: 2, name: 'Main UPS' },
                    { position: 6, type: 'storage', size: 2, name: 'Primary Storage' },
                    { position: 10, type: 'server', size: 2, name: 'Virtualization Host 1' },
                    { position: 12, type: 'server', size: 2, name: 'Virtualization Host 2' },
                    { position: 41, type: 'network', size: 1, name: 'Core Switch' },
                    { position: 42, type: 'network', size: 1, name: 'Access Switch' }
                ]
            },
            compute: {
                name: "High-Performance Compute Cluster",
                description: `Designed for computational workloads:
                • Dense compute configuration
                • High-power UPS system
                • Minimal storage footprint
                • High-performance networking

                Key design principles:
                • Maximum compute density
                • Efficient power distribution
                • High-speed interconnect
                • Optimized airflow design`,
                config: [
                    { position: 2, type: 'ups', size: 4, name: 'High Capacity UPS' },
                    { position: 8, type: 'storage', size: 2, name: 'Scratch Storage' },
                    { position: 12, type: 'server', size: 2, name: 'Head Node' },
                    { position: 14, type: 'server', size: 1, name: 'Compute Node 1' },
                    { position: 15, type: 'server', size: 1, name: 'Compute Node 2' },
                    { position: 16, type: 'server', size: 1, name: 'Compute Node 3' },
                    { position: 17, type: 'server', size: 1, name: 'Compute Node 4' },
                    { position: 18, type: 'server', size: 1, name: 'Compute Node 5' },
                    { position: 19, type: 'server', size: 1, name: 'Compute Node 6' },
                    { position: 20, type: 'server', size: 1, name: 'Compute Node 7' },
                    { position: 21, type: 'server', size: 1, name: 'Compute Node 8' },
                    { position: 41, type: 'network', size: 1, name: 'Management Switch' },
                    { position: 42, type: 'network', size: 1, name: 'High-Speed Switch' }
                ]
            },
            storage: {
                name: "Storage-Focused Configuration",
                description: `Optimized for high-capacity storage:
                • Multiple storage arrays
                • Redundant storage controllers
                • Dedicated backup systems
                • Separate storage network

                Key design principles:
                • Maximum storage density
                • Data protection focus
                • Separate storage network
                • Backup integration`,
                config: [
                    { position: 2, type: 'ups', size: 2, name: 'Primary UPS' },
                    { position: 4, type: 'ups', size: 2, name: 'Secondary UPS' },
                    { position: 8, type: 'storage', size: 4, name: 'Primary Storage Array 1' },
                    { position: 12, type: 'storage', size: 4, name: 'Primary Storage Array 2' },
                    { position: 16, type: 'storage', size: 4, name: 'Backup Storage Array' },
                    { position: 22, type: 'server', size: 2, name: 'Storage Controller 1' },
                    { position: 24, type: 'server', size: 2, name: 'Storage Controller 2' },
                    { position: 26, type: 'server', size: 2, name: 'Backup Server' },
                    { position: 40, type: 'network', size: 1, name: 'Management Switch' },
                    { position: 41, type: 'network', size: 1, name: 'Storage Switch A' },
                    { position: 42, type: 'network', size: 1, name: 'Storage Switch B' }
                ]
            }
        };

        function initRack() {
            const rack = document.getElementById('rack');
            rack.innerHTML = '';

            for (let i = RACK_UNITS; i > 0; i--) {
                const unit = document.createElement('div');
                unit.className = 'rack-unit';
                unit.setAttribute('data-u', i);
                unit.addEventListener('click', () => addEquipment(i));
                rack.appendChild(unit);
            }
            updateRackView();
        }

        function selectEquipment(type, size, defaultName) {
            selectedConfig = { type, size, defaultName };
            document.querySelectorAll('button').forEach(btn => btn.classList.remove('selected'));
            event.target.classList.add('selected');
        }

        function addEquipment(position) {
            if (!selectedConfig) return;

            // Validate position is within bounds
            if (position < 1 || position > RACK_UNITS) return;

            // Check if there's enough continuous space
            for (let i = 0; i < selectedConfig.size; i++) {
                const checkPosition = position - i;
                if (checkPosition <= 0) {
                    alert('Not enough space at the bottom of the rack');
                    return;
                }
                if (rackContents[checkPosition - 1] !== null) {
                    alert('Space already occupied');
                    return;
                }
            }

            const name = prompt('Enter equipment name:', `${selectedConfig.defaultName} ${position}`);
            if (!name) return;

            // Add equipment
            const equipment = {
                type: selectedConfig.type,
                size: selectedConfig.size,
                name: name,
                position: position,
                startUnit: position - selectedConfig.size + 1
            };

            // Fill rack contents from bottom to top of the equipment
            for (let i = 0; i < selectedConfig.size; i++) {
                rackContents[position - i - 1] = equipment;
            }

            updateRackView();
        }

        function updateRackView() {
            const rackUnits = document.querySelectorAll('.rack-unit');
            rackUnits.forEach((unit, index) => {
                unit.innerHTML = '';
                if (rackContents[index] !== null) {
                    const equipment = rackContents[index];
                    if (index === 0 || rackContents[index - 1]?.name !== equipment.name) {
                        const div = document.createElement('div');
                        div.className = `equipment type-${equipment.type}`;
                        div.style.height = `${equipment.size * 20 + (equipment.size - 1) * 2}px`;

                        const nameSpan = document.createElement('div');
                        nameSpan.className = 'equipment-name';
                        nameSpan.textContent = equipment.name;

                        const typeSpan = document.createElement('div');
                        typeSpan.className = 'equipment-type';
                        typeSpan.textContent = `${equipment.size}U ${equipment.type}`;

                        div.appendChild(nameSpan);
                        div.appendChild(typeSpan);
                        unit.appendChild(div);
                    }
                }
            });
        }

        function loadSampleConfig(configType) {
            clearRack();
            const config = configurations[configType];

            // Display configuration description
            const descriptionDiv = document.getElementById('config-description');
            const configText = document.getElementById('config-text');
            descriptionDiv.style.display = 'block';
            configText.innerHTML = `<h4>${config.name}</h4><pre style="white-space: pre-wrap;">${config.description}</pre>`;

            config.config.forEach(equipment => {
                // Fill the rack contents array
                for (let i = 0; i < equipment.size; i++) {
                    rackContents[equipment.position - i - 1] = equipment;
                }
            });

            updateRackView();
        }

        function clearRack() {
            rackContents.fill(null);
            selectedConfig = null;
            document.querySelectorAll('button').forEach(btn => btn.classList.remove('selected'));
            const descriptionDiv = document.getElementById('config-description');
            descriptionDiv.style.display = 'none';
            updateRackView();
        }

        // Initialize the rack when the page loads
        initRack();